In [1]:
import tensorflow as tf
import numpy as np
import os
import re

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
# PARAMETERS
img_width, img_height = 224, 224
image_size = img_width* img_height
LEARNING_RATE = 0.00001
EPSILON = 1e-4
LABEL_NUM = 2
TRAIN_DATASET = 10000
TEST_DATASET = 2500
BATCH_SIZE = 32
epoch = 2000

In [3]:
# Get data directory
def read_data(data_dir):
    data_directory = data_dir
    
    filename ={'train':[],'validation':[]}
    dataset = ['train','validation']

    for dataset_name in dataset:
        for subFile in os.listdir(data_directory):
            if dataset_name in subFile and re.match(r'^.*\.tfrecords$', subFile):
                filename[dataset_name].append(data_directory+"/"+subFile)
#     print(filename)        
#     training_filenames = filename['train']
#     validation_filenames = filename['validation']
    
    return filename['train'] , filename['validation'] 

In [4]:
def parse(serialized):
    features = { 'label': tf.FixedLenFeature([], tf.int64),
               'text_label': tf.FixedLenFeature([], tf.string),
               'image': tf.FixedLenFeature([], tf.string)
               }

    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)

    image_raw = parsed_example['image']

    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.reshape(image, [img_width, img_height, 3])
#     image = tf.image.resize_images(
#                 image,
#                 [64, 64]
#             )
#     image = tf.image.rgb_to_grayscale(image, name=None)
    image = tf.cast(image, tf.float32)    
    image = image / 255.
    
    label = parsed_example['label']
    label = tf.one_hot(label,LABEL_NUM)
    label = tf.cast(label, tf.float32)
#     label = tf.reshape(label, [1])
    
    return image, label

In [5]:
def input_pipeline(filename , BUFFER_SIZE = 25000):
    dataset = tf.data.TFRecordDataset(filename)
    dataset = dataset.map(parse)
    dataset = dataset.shuffle(buffer_size=BUFFER_SIZE)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_one_shot_iterator()
    return iterator


In [6]:
class CNN():
    def __init__(self):
        self.cost = None
        self.accuracy = None
        self.prediction = None
        self.logits = None
        self.global_step = None 
        
        
#         print("CNN")

    def predict(self, x):
        
        #model VGG16
        x = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])

        x = tf.layers.conv2d(inputs = x,filters = 128,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 128,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])
        
        x = tf.layers.conv2d(inputs = x,filters = 256,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 256,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 256,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])
        
        x = tf.layers.conv2d(inputs = x,filters = 512,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 512,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 512,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])
        
        x = tf.layers.conv2d(inputs = x,filters = 512,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 512,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 512,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])
        
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, 4096, activation=tf.nn.relu)       
#         x = tf.layers.dropout(x,rate=0.5)
        x = tf.layers.dense(x, 4096, activation=tf.nn.relu)
        x = tf.layers.dropout(x,rate=0.5)
        x = tf.layers.dense(x, 2, activation=tf.nn.sigmoid, use_bias = True)
        
        return x
    
    def train(self, x, y, learning_rate = 0.01, epsilon = 1e-10):
        self.logits = self.predict(x)
        
        with tf.name_scope("cost"):
            self.cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = self.logits)  , name='cost')
            tf.summary.scalar("cost", self.cost)
        
        self.global_step = tf.train.get_or_create_global_step()   
        optimizer = tf.train.RMSPropOptimizer(learning_rate = learning_rate, epsilon=epsilon)
        optimize = optimizer.minimize(self.cost , global_step = self.global_step)
        accuracy = self.validate(self.logits, y)
        return self.cost, accuracy, optimize, self.global_step
        
    def validate(self, logits, y):         
        with tf.name_scope("validate"):
            correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar("accuracy", self.accuracy )
            tf.summary.histogram("accuracy", self.accuracy )
            
        return self.accuracy
    
    def test(self, x, y):
        self.logits = self.predict(x)
        accuracy = validate(self.logits, y)
        return accuracy
        

 

In [7]:
tf.reset_default_graph()

In [8]:
data_dir = "../data"    
training_filenames, validation_filenames= read_data(data_dir)
train_dataset = input_pipeline(training_filenames)
test_dataset = input_pipeline(validation_filenames)

print(training_filenames)

['../data/train-3.tfrecords', '../data/train-2.tfrecords', '../data/train-1.tfrecords', '../data/train-0.tfrecords']


In [9]:
clf = CNN()
x_train, y_train = train_dataset.get_next()
x_test, y_test = train_dataset.get_next()

cost , accuracy, optimize, global_step = clf.train(x = x_train, y = y_train, learning_rate = LEARNING_RATE, epsilon = EPSILON)
test_accuracy = clf.validate(clf.predict(x_test), y_test)
merge = tf.summary.merge_all()

In [ ]:
ACC_TEST , ACC_TRAIN = [],[]
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('./logs/train', sess.graph)
    test_writer =  tf.summary.FileWriter('./logs/test', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for i in range(epoch):        
        total_loss, total_acc = 0 ,0
        for x in range(0, (TRAIN_DATASET//BATCH_SIZE)* BATCH_SIZE, BATCH_SIZE):            
            if x == 0:
                loss, acc, _, summary, global_step_val= sess.run([cost, accuracy, optimize, merge, global_step])
                train_writer.add_summary(summary, global_step_val)    
            else:
                loss, acc, _= sess.run([cost, accuracy, optimize])
                
            total_loss += loss
            total_acc += acc    
        

        total_loss /= (TRAIN_DATASET // BATCH_SIZE) 
        total_acc /= (TRAIN_DATASET // BATCH_SIZE)
        ACC_TRAIN.append(total_acc)
        
        total_acc = 0 
        for x in range(0, (TEST_DATASET//BATCH_SIZE)* BATCH_SIZE, BATCH_SIZE):
            acc, summary, global_step_val= sess.run([test_accuracy, merge, global_step])
            test_writer.add_summary(summary, global_step_val)   
                
            total_acc += acc    
        
        total_acc /= (TEST_DATASET // BATCH_SIZE)
        ACC_TEST.append(total_acc)
        print("Epoch : %d | TRAIN Loss %f , Accuracy %f | TEST Accuracy %f"% ((i+1),total_loss, ACC_TRAIN[-1], ACC_TEST[-1]))  
        

Epoch : 1 | TRAIN Loss 0.723904 , Accuracy 0.503906 | TEST Accuracy 0.496795
Epoch : 2 | TRAIN Loss 0.723545 , Accuracy 0.492989 | TEST Accuracy 0.483974
Epoch : 3 | TRAIN Loss 0.723177 , Accuracy 0.518429 | TEST Accuracy 0.501603
Epoch : 4 | TRAIN Loss 0.722819 , Accuracy 0.513922 | TEST Accuracy 0.502404
Epoch : 5 | TRAIN Loss 0.722456 , Accuracy 0.507712 | TEST Accuracy 0.506811
Epoch : 6 | TRAIN Loss 0.722082 , Accuracy 0.508013 | TEST Accuracy 0.509215
Epoch : 7 | TRAIN Loss 0.721695 , Accuracy 0.503105 | TEST Accuracy 0.515625
Epoch : 8 | TRAIN Loss 0.721285 , Accuracy 0.506611 | TEST Accuracy 0.479567
Epoch : 9 | TRAIN Loss 0.720859 , Accuracy 0.497496 | TEST Accuracy 0.501603
Epoch : 10 | TRAIN Loss 0.720387 , Accuracy 0.511518 | TEST Accuracy 0.506811
Epoch : 11 | TRAIN Loss 0.719900 , Accuracy 0.499499 | TEST Accuracy 0.495593
Epoch : 12 | TRAIN Loss 0.719358 , Accuracy 0.498397 | TEST Accuracy 0.499199
Epoch : 13 | TRAIN Loss 0.718751 , Accuracy 0.502204 | TEST Accuracy 0.50

Epoch : 107 | TRAIN Loss 0.692431 , Accuracy 0.499399 | TEST Accuracy 0.501202
Epoch : 108 | TRAIN Loss 0.692368 , Accuracy 0.499900 | TEST Accuracy 0.505609
Epoch : 109 | TRAIN Loss 0.692435 , Accuracy 0.497796 | TEST Accuracy 0.490785
Epoch : 110 | TRAIN Loss 0.692135 , Accuracy 0.501903 | TEST Accuracy 0.510817
Epoch : 111 | TRAIN Loss 0.692656 , Accuracy 0.495493 | TEST Accuracy 0.517228
Epoch : 112 | TRAIN Loss 0.692242 , Accuracy 0.500200 | TEST Accuracy 0.501202
Epoch : 113 | TRAIN Loss 0.692140 , Accuracy 0.503105 | TEST Accuracy 0.507212
Epoch : 114 | TRAIN Loss 0.692442 , Accuracy 0.492188 | TEST Accuracy 0.504006
Epoch : 115 | TRAIN Loss 0.691826 , Accuracy 0.505208 | TEST Accuracy 0.502404
Epoch : 116 | TRAIN Loss 0.692230 , Accuracy 0.497696 | TEST Accuracy 0.509615
Epoch : 117 | TRAIN Loss 0.692176 , Accuracy 0.497696 | TEST Accuracy 0.505208
Epoch : 118 | TRAIN Loss 0.691824 , Accuracy 0.499499 | TEST Accuracy 0.508013
Epoch : 119 | TRAIN Loss 0.692003 , Accuracy 0.49669

Epoch : 211 | TRAIN Loss 0.683653 , Accuracy 0.499700 | TEST Accuracy 0.520032
Epoch : 212 | TRAIN Loss 0.684332 , Accuracy 0.498097 | TEST Accuracy 0.508814
Epoch : 213 | TRAIN Loss 0.683403 , Accuracy 0.492087 | TEST Accuracy 0.507612
Epoch : 214 | TRAIN Loss 0.683430 , Accuracy 0.501502 | TEST Accuracy 0.502003
Epoch : 215 | TRAIN Loss 0.683797 , Accuracy 0.497396 | TEST Accuracy 0.500000
Epoch : 216 | TRAIN Loss 0.682458 , Accuracy 0.500901 | TEST Accuracy 0.507612
Epoch : 217 | TRAIN Loss 0.682215 , Accuracy 0.505108 | TEST Accuracy 0.485577
Epoch : 218 | TRAIN Loss 0.684170 , Accuracy 0.497997 | TEST Accuracy 0.513622
Epoch : 219 | TRAIN Loss 0.682789 , Accuracy 0.500401 | TEST Accuracy 0.516026
Epoch : 220 | TRAIN Loss 0.682306 , Accuracy 0.499199 | TEST Accuracy 0.509615
Epoch : 221 | TRAIN Loss 0.683106 , Accuracy 0.499199 | TEST Accuracy 0.506811
Epoch : 222 | TRAIN Loss 0.683437 , Accuracy 0.496795 | TEST Accuracy 0.510016
Epoch : 223 | TRAIN Loss 0.682879 , Accuracy 0.50010

Epoch : 315 | TRAIN Loss 0.679847 , Accuracy 0.496094 | TEST Accuracy 0.505208
Epoch : 316 | TRAIN Loss 0.679318 , Accuracy 0.501603 | TEST Accuracy 0.497596
Epoch : 317 | TRAIN Loss 0.679140 , Accuracy 0.501102 | TEST Accuracy 0.492788
Epoch : 318 | TRAIN Loss 0.677732 , Accuracy 0.498898 | TEST Accuracy 0.493189
Epoch : 319 | TRAIN Loss 0.678110 , Accuracy 0.503405 | TEST Accuracy 0.500801
Epoch : 320 | TRAIN Loss 0.679192 , Accuracy 0.497196 | TEST Accuracy 0.497997
Epoch : 321 | TRAIN Loss 0.677873 , Accuracy 0.503305 | TEST Accuracy 0.488381
Epoch : 322 | TRAIN Loss 0.678582 , Accuracy 0.499800 | TEST Accuracy 0.510016
Epoch : 323 | TRAIN Loss 0.677700 , Accuracy 0.500601 | TEST Accuracy 0.488782
Epoch : 324 | TRAIN Loss 0.677873 , Accuracy 0.505308 | TEST Accuracy 0.505609
Epoch : 325 | TRAIN Loss 0.678583 , Accuracy 0.498998 | TEST Accuracy 0.503205
Epoch : 326 | TRAIN Loss 0.678297 , Accuracy 0.500601 | TEST Accuracy 0.492788
Epoch : 327 | TRAIN Loss 0.678216 , Accuracy 0.50210

Epoch : 419 | TRAIN Loss 0.669909 , Accuracy 0.491987 | TEST Accuracy 0.494792
Epoch : 420 | TRAIN Loss 0.668476 , Accuracy 0.501603 | TEST Accuracy 0.504407
Epoch : 421 | TRAIN Loss 0.667255 , Accuracy 0.504708 | TEST Accuracy 0.487981
Epoch : 422 | TRAIN Loss 0.668227 , Accuracy 0.492889 | TEST Accuracy 0.498798
Epoch : 423 | TRAIN Loss 0.667770 , Accuracy 0.506611 | TEST Accuracy 0.495192
Epoch : 424 | TRAIN Loss 0.667688 , Accuracy 0.501903 | TEST Accuracy 0.481971
Epoch : 425 | TRAIN Loss 0.668152 , Accuracy 0.502704 | TEST Accuracy 0.489183
Epoch : 426 | TRAIN Loss 0.667101 , Accuracy 0.503606 | TEST Accuracy 0.500401
Epoch : 427 | TRAIN Loss 0.667696 , Accuracy 0.497897 | TEST Accuracy 0.506811
Epoch : 428 | TRAIN Loss 0.666921 , Accuracy 0.493790 | TEST Accuracy 0.505609
Epoch : 429 | TRAIN Loss 0.667880 , Accuracy 0.500701 | TEST Accuracy 0.495593
Epoch : 430 | TRAIN Loss 0.667504 , Accuracy 0.497997 | TEST Accuracy 0.495994
Epoch : 431 | TRAIN Loss 0.667513 , Accuracy 0.49949

Epoch : 523 | TRAIN Loss 0.649486 , Accuracy 0.498898 | TEST Accuracy 0.497196
Epoch : 524 | TRAIN Loss 0.648937 , Accuracy 0.496595 | TEST Accuracy 0.506010
Epoch : 525 | TRAIN Loss 0.646956 , Accuracy 0.501402 | TEST Accuracy 0.505208
Epoch : 526 | TRAIN Loss 0.649788 , Accuracy 0.492288 | TEST Accuracy 0.506811
Epoch : 527 | TRAIN Loss 0.647715 , Accuracy 0.496895 | TEST Accuracy 0.499599
Epoch : 528 | TRAIN Loss 0.647274 , Accuracy 0.501302 | TEST Accuracy 0.496795
Epoch : 529 | TRAIN Loss 0.647109 , Accuracy 0.500901 | TEST Accuracy 0.499199
Epoch : 530 | TRAIN Loss 0.646794 , Accuracy 0.499299 | TEST Accuracy 0.489183
Epoch : 531 | TRAIN Loss 0.648386 , Accuracy 0.500801 | TEST Accuracy 0.517628
Epoch : 532 | TRAIN Loss 0.645790 , Accuracy 0.497997 | TEST Accuracy 0.498798
Epoch : 533 | TRAIN Loss 0.643898 , Accuracy 0.500200 | TEST Accuracy 0.483173
Epoch : 534 | TRAIN Loss 0.646100 , Accuracy 0.504107 | TEST Accuracy 0.488381
Epoch : 535 | TRAIN Loss 0.645769 , Accuracy 0.50050